In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
import scipy.stats
import scipy.optimize

import numpy as np
import matplotlib.pyplot as plt
import uproot
import scipy.optimize
import matplotlib.gridspec as gridspec


import awkward

from matplotlib.colors import LogNorm


import math

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

energy_range = (0.03,2.03)
energy_bins = 25

def gauss_exp(x, n, mu, sigma, k):
    sigma = abs(sigma)
    condition = (x - mu) / sigma >= -k    
    y = np.copy(x)
    y[condition] = n * np.exp(-0.5 * ((x[condition] - mu) / sigma)**2)
    y[~condition] = n * np.exp(k**2 / 2 + k * ((x[~condition] - mu) / sigma))
#     print(x)
    return y

def get_function_max(f, *args):
    def func(x, *arg):
        return -f(x, *arg)
    return f(scipy.optimize.fmin(func, 0, args=args, disp=False)[0], *args)

def find_nearest(array, value):
    array = np.asarray(array)
    #print (array)
    idx = (np.abs(array - value)).argmin()
    return idx

def mpv(array):
    if sum(array) < 5:
        return np.median(array)
    
    n_bins = energy_bins
    r = energy_range
    hist, bin_edges = np.histogram(array, bins=n_bins, range=r)
    
    bin_centers = [i*r[1]/n_bins-r[1]/(n_bins*2) for i in range(1,n_bins+1)]
    try:
        popt, pcov = scipy.optimize.curve_fit(gauss_exp, bin_centers, hist, maxfev=10000)
        return scipy.optimize.fmin(lambda x: -gauss_exp(x, *popt), 0)
    except RuntimeError:
        return np.median(array)
    
def fwhm(array):
    if sum(array) < 5:
        return np.std(array)

    n_bins = energy_bins
    r = energy_range
    hist, bin_edges = np.histogram(array, bins=n_bins, range=r)
    
    bin_centers = [i*r[1]/n_bins-r[1]/(n_bins*2) for i in range(1,n_bins+1)]

    try:
        popt, pcov = scipy.optimize.curve_fit(gauss_exp, bin_centers, hist, maxfev=10000)
        x_values = np.linspace(r[0], r[1], 1000)
        y_values = gauss_exp(np.linspace(r[0], r[1], 1000), *popt)
        try:
            x_max = scipy.optimize.fmin(lambda x: -gauss_exp(x, *popt), 0)
        except RuntimeError:
            x_max = np.median(array)
        y_max = find_nearest(y_values, gauss_exp(x_max, *popt))
        y_max_value = y_values[y_max]
        fwhm1 = find_nearest(y_values[:y_max], y_max_value/2)
        fwhm2 = find_nearest(y_values[y_max:], y_max_value/2)
        x_2 = x_values[y_max:][fwhm2]     
        x_1 = x_values[:y_max][fwhm1]
        return x_2-x_1
    except RuntimeError:
        return np.std(array)

In [ ]:

plt.rcParams.update({'font.size': 16})

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"
NUE = 'allnues'
#NUE = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2"


UPROOTDF = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

variables = ["elec_e", "shr_energy_tot_cali", "selected","nslice",\
             "trk_energy_tot","proton_e","n_tracks_contained",\
             "trk_llr_pid_score_v","trk_id",\
             "trk_energy_muon","muon_e",
             "topological_score",'nu_e','true_e_visible',
            "reco_nu_vtx_sce_y","reco_nu_vtx_sce_z","trk_len",
             "shr_energy_tot", "reco_nu_vtx_sce_x", "isVtxInFiducial"]

df = UPROOTDF.pandas.df(variables, flatten=False)

df['proton_ke'] = df['proton_e']-0.938
df['muon_ke'] = df['muon_e']-0.105

df['reco_e'] = (df["shr_energy_tot_cali"]) / 0.79 + df["trk_energy_tot"]

trk_llr_pid_v = UPROOTDF.array('trk_llr_pid_score_v')
trk_id = UPROOTDF.array('trk_id')-1 # I think we need this -1 to get the right result
trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
df['trkpid'] = trk_llr_pid_v_sel


QUERY = 'nslice == 1'
QUERY += ' and selected == 1'
QUERY += ' and n_tracks_contained > 0'
QUERY += ' and shr_energy_tot_cali > 0.07'
QUERY += ' and isVtxInFiducial == 1'
QUERY += ' and trkpid < -0.02'

# numu selection
# muon selection
#QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
#QUERY += ' and topological_score > 0.06'
#QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
#QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
#QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
#QUERY += ' and trkpid > 0.2'

df = df.query(QUERY)


In [ ]:
# define resolution to be measured
RECOVAR = 'shr_energy_tot_cali'
TRUEVAR = 'elec_e'
EMIN = 0.03
EMAX = 2.03
NBINS = 10
EBINS = np.linspace(EMIN,EMAX,NBINS+1)
RESLOW = -1
RESHIGH = 1
VARIABLENAME = 'Electron Energy'

RECOVAR = 'trk_energy_tot'
TRUEVAR = 'proton_ke'
EMIN = 0.05
EMAX = 0.40
NBINS = 20
EBINS = np.linspace(EMIN,EMAX,NBINS+1)
RESLOW = -0.4
RESHIGH = 0.1
VARIABLENAME = 'Proton KE'

#RECOVAR = 'trk_energy_muon'
#TRUEVAR = 'muon_ke'
#EMIN = 0.05
#EMAX = 1.00
#NBINS = 20
#EBINS = np.linspace(EMIN,EMAX,NBINS+1)
#RESLOW = -0.5
#RESHIGH = 0.5
#VARIABLENAME = 'Muon KE'

RECOVAR = 'reco_e'
TRUEVAR = 'true_e_visible'
EMIN = 0.2
EMAX = 1.5
NBINS = 14
EBINS = np.linspace(EMIN,EMAX,NBINS+1)
RESLOW = -1
RESHIGH = 1
VARIABLENAME = 'Neutrino Energy'

In [ ]:
bin_means, bin_edges, binnumber = scipy.stats.binned_statistic(
    df[TRUEVAR], df[RECOVAR], statistic=mpv, range=(EMIN,EMAX), bins=EBINS)

#bin_stdev, bin_edges, binnumber = scipy.stats.binned_statistic(
#    df[TRUEVAR], df[RECOVAR], statistic=fwhm, range=(EMIN,EMAX), bins=10)

In [ ]:
heatmap, xedges, yedges = np.histogram2d(df[TRUEVAR], df[RECOVAR], bins=200)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
fig, ax  = plt.subplots(figsize=(7,7))
ax.imshow(heatmap.T, extent=extent, origin='lower')#,norm=LogNorm())
bin_centers = (bin_edges + (bin_edges[1]-bin_edges[0])/2)[:-1]
#ax.errorbar(
#    bin_centers,
#    bin_means,
#    xerr=0.01,
#    #yerr=bin_stdev/2,
#    fmt='ko')

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(bin_centers, bin_means)
#ax.plot(bin_centers, intercept + slope*bin_centers, 'r', label=r'$E^{\mathrm{e}}_{\mathrm{reco}}=%.2f E^{\mathrm{e}} %.2f$ MeV' % (slope, intercept*1000))
#leg = ax.legend(title=r'MicroBooNE Simulation Preliminary')
#leg._legend_box.align = "left"
plt.setp(leg.get_title(), fontweight='bold')


ax.set_xlabel(r"$E^{\mathrm{\nu_e}}_{\mathrm{visible}}$  [GeV]")
ax.set_ylabel(r"$E^{\mathrm{\nu_e}}_{\mathrm{reco}}$ [GeV]")
ax.set_ylim(EMIN,EMAX)
ax.set_xlim(EMIN,EMAX)

fig.savefig(ls.plots_path+"e_calib.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 20))
gs = gridspec.GridSpec(int(NBINS/2), 2, hspace=0.8, wspace=0.4)

params = {
    'legend.fontsize': 'large',
    'axes.labelsize': 'small',
    'axes.titlesize': 'small',
    'xtick.labelsize': 'small',
    'ytick.labelsize': 'small'
}

pylab.rcParams.update(params)

for i in range(NBINS):
    df_bin = df.query("%f < %s < %f" % (EBINS[i],TRUEVAR,EBINS[i+1]))
    label_true = ""
    label_reco = ""
    label_fit = ""
    if i == 0:
        label_true = r"$E^e$"
        label_reco = r"$E^e_{reco}$"
        label_fit = r'GaussExp fit'

    plt.subplot(gs[i]).hist(
        df_bin[TRUEVAR],
        linewidth=1.5,
        bins=100,
        range=(EMIN,EMAX),
        histtype='step',
        color='k',
        label=label_true)

    hist, bin_edges, patches = plt.subplot(gs[i]).hist(
        df_bin[RECOVAR],
        linewidth=1.5,
        color='b',
        histtype='step',
        range=(EMIN,EMAX),
        bins=100,
        label=label_reco)

    r = (EMIN,EMAX)
    n_bins = 100

    bin_centers = [
        i * (r[1] - r[0]) / n_bins - (r[1] - r[0]) / (n_bins * 2) + r[0]
        for i in range(1, n_bins + 1)
    ]
    
    #print (bin_centers)
    
    popt, pcov = scipy.optimize.curve_fit(
        gauss_exp, bin_centers, hist, maxfev=10000)
    x_values = np.linspace(r[0], r[1], 1000)
    y_values = gauss_exp(np.linspace(r[0], r[1], 1000), *popt)
    plt.plot(x_values, y_values, color='r', label=label_fit)
    plt.subplot(gs[i]).set_xlim(EMIN,EMAX)
    plt.subplot(gs[i]).set_title(
        "%g MeV < $E^e$ < %g MeV" % (EBINS[i] * 1000,EBINS[i+1] * 1000),
        fontweight='bold')
    plt.subplot(gs[i]).set_xlabel(r"$E^e$ [GeV]")
    plt.subplot(gs[i]).set_ylabel(r"N. Entries / 0.02 GeV")

fig.legend(loc='upper center', ncol=3, frameon=False)
fig.tight_layout()
fig.subplots_adjust(top=0.93)
fig.savefig(ls.plots_path+"e_spectra.pdf")

In [ ]:
params = {
    'legend.fontsize': 'large',
    'axes.labelsize': 'small',
    'axes.titlesize': 'small',
    'xtick.labelsize': 'small',
    'ytick.labelsize': 'small'
}

pylab.rcParams.update(params)

fig_res, ax_res = plt.subplots(figsize=(10, 20))
gs_res = gridspec.GridSpec(int(NBINS/2), 2, hspace=0.8, wspace=0.4)

sigma = np.array([])
sigma_err = np.array([])

for i in range(NBINS):
    df_bin = df.query("%f < %s < %f" %(EBINS[i],TRUEVAR,EBINS[i+1]))
    label_true = ""
    label_reco = ""
    label_fit = ""
    if i == 0:
        label_true = r"$E^e$"
        label_reco = r"$E^e_{reco}$"
        label_fit = r'GaussExp fit'

    e_res = (df_bin[RECOVAR] - df_bin[TRUEVAR]) / df_bin[TRUEVAR]
    #e_res = ( (df_bin[RECOVAR]/slope) - df_bin[TRUEVAR]) / df_bin[TRUEVAR]
    n_bins = 25

    hist, bin_edges, patches = plt.subplot(gs_res[i]).hist(
        e_res,
        linewidth=1.5,
        bins=n_bins,
        range=(RESLOW,RESHIGH),
        histtype='step',
        color='k',
        label=label_true)

    r = (RESLOW,RESHIGH)

    bin_centers = np.array([
        i * (r[1] - r[0]) / len(hist) - (r[1] - r[0]) / (len(hist) * 2) + r[0]
        for i in range(1, len(hist) + 1)
    ])

    #'''
    fit_range = np.logical_and(bin_centers < 0.5, bin_centers > -0.5)
    # n, mu, sigma, k
    popt, pcov = scipy.optimize.curve_fit(
        gauss_exp,
        bin_centers[fit_range],
        hist[fit_range],
        maxfev=10000,
        p0=(100,0.0,0.2,0.1),
        bounds=((0, -0.5, 0, 0), (1e4, 0.1, 1.0, 1)))
    
    print ('fit-values are ',popt)

    x_values = np.linspace(r[0], r[1], 1000)
    y_values = gauss_exp(np.linspace(r[0], r[1], 1000), *popt)
    sigma = np.append(sigma, popt[2])
    sigma_err = np.append(sigma_err, math.sqrt(np.diagonal(pcov)[2]))
    plt.plot(x_values, y_values, color='r', label=label_fit)
    plt.subplot(gs_res[i]).set_xlim(RESLOW,RESHIGH)
    plt.subplot(gs_res[i]).set_title(
        "%g MeV < $E^e$ < %g MeV" % (EBINS[i]*1000.,EBINS[i+1]*1000.),\
        fontweight="bold")
    plt.subplot(gs_res[i]).set_xlabel(r"$(E^e_{corr}-E^e)/E^e$")
    plt.subplot(gs_res[i]).set_ylabel(r"N. Entries / 0.02")
    #'''

fig_res.legend(loc='upper center', ncol=2, frameon=False)
fig_res.tight_layout()
fig_res.subplots_adjust(top=0.93)
fig_res.savefig(ls.plots_path+"res.pdf")

In [ ]:
params = {
    'legend.fontsize': 'large',
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

fig, ax = plt.subplots(figsize=(6, 6))
x_centers = np.array([EBINS[i] * 1000 for i in range(NBINS)])


def res_fit(x, a, b, c):
    return np.sqrt((a / np.sqrt(x))**2 + (b / x)**2 + c**2)


print(list(sigma))

ax.errorbar(
    x_centers / 1000,
    sigma * 100,
    #xerr=0.1,
    yerr=sigma_err[0:] * 100,
    fmt='o',
    label=r"")

popt, pcov = scipy.optimize.curve_fit(
    res_fit,
    x_centers / 1000,
    sigma * 100,
    maxfev=10000,
    bounds=((0, 0, 0), (np.inf, np.inf, np.inf)),
    sigma=sigma_err * 100)


x_values = np.linspace(EMIN, EMAX, 1000)
y_values = res_fit(np.linspace(EMIN, EMAX, 1000), *popt)

plt.plot(
    x_values,
    y_values,
    linewidth=2,
    label=
    r'$\left(\frac{%.2f}{\sqrt{E / \mathrm{GeV}}}\oplus\frac{%.2f}{E / \mathrm{GeV}}\oplus %.2f\right)$ %%'
    % (popt[0], popt[1], popt[2]))

ax.set_ylim(0,20)
ax.set_xlim(EMIN, EMAX)
ax.set_xlabel(r"%s [GeV]"%VARIABLENAME,fontsize=22)
ax.set_ylabel(r"$\sigma$ [%]",fontsize=22)
plt.grid()

fig.legend(frameon=False, loc='best', bbox_to_anchor=(0.45, 0.45, 0.45, 0.45),fontsize=18)
fig.tight_layout()
fig.savefig(ls.plots_path+"sigma_res.pdf")